In [1]:
from scipy import stats 

In [5]:
import numpy as np
fileloc='data/'
p=np.zeros((56, nof))
for i in range(1,57):
    fname=fileloc+str(i)+'.csv'
    data=np.genfromtxt(fname,delimiter=',')
    lab=data[:,-1]
    fet=data[:,0:-1]
    nof=np.shape(fet)[1]
    id1=np.where(lab==0)
    id2=np.where(lab>0)
    
    for j in range(0,nof):
        fv=data[:,j]
        fv0=fv[id1[0]]
        fv1=fv[id2[0]]
        w,p[i-1,j]= stats.ranksums(fv0,fv1)
    in1=np.where(p[i-1,:]<=0.05)
    datan=data[:,in1[0]]
    data=np.concatenate((datan,lab.reshape(-1,1)), axis=1)
    fname=fileloc+str(56+i)+".csv"
    np.savetxt(fname,data, delimiter=',', fmt='%f')

In [6]:
fname=fileloc+'pvalue.csv'
np.savetxt(fname,p, delimiter=',', fmt='%f')

In [27]:
from sklearn.decomposition import PCA
for i in range(1,57):
    fname=fileloc+str(i)+'.csv'
    df=np.genfromtxt(fname,delimiter=',')
    pca = PCA(n_components=5)
    pca.fit(df[:,0:-1])
    data=pca.transform(df[:,0:-1])
    data=np.concatenate((data,df[:,-1].reshape(-1,1)), axis=1)
    fname=fileloc+str(56*2+ i)+".csv"
    np.savetxt(fname,data, delimiter=',', fmt='%f')

In [8]:
from sklearn.cluster import KMeans
def ginical(fv,lab):
    kmeans = KMeans(n_clusters=2, random_state=0).fit(fv.reshape(-1, 1))
    cl=kmeans.labels_
    in1=np.where(cl==0)
    in2=np.where(cl==1)
    if len(in1[0])==0 or len(in2[0])==0:
        return 0.5
    lab0=lab[in1[0]]
    lab1=lab[in2[0]]
    n00=len(np.where(lab0==0)[0])
    n01=len(np.where(lab0>0)[0])
    n10=len(np.where(lab1==0)[0])
    n11=len(np.where(lab1>0)[0])
    p00=n00/(n00+n01)
    p01=n01/(n00+n01)
    p10=n10/(n10+n11)
    p11=n11/(n10+n11)
    p0=1-p00*p00-p01*p01
    p1=1-p01*p01-p11*p11
    gv=(len(lab0)/len(cl))*p0+(len(lab1)/len(cl))*p1
    return gv

In [28]:
for i in range(1,57):
    fname=fileloc+str(i)+'.csv'
    data=np.genfromtxt(fname,delimiter=',')
    nof=np.shape(data[:,0:-1])[1]
    gv=np.zeros((nof))
    for j in range(0,nof):
        gv[j]=ginical(data[:,j],data[:,-1])
    b=np.argsort(gv)
    datan=data[:,b[0:5]]
    data=np.concatenate((datan,data[:,-1].reshape(-1,1)), axis=1)
    fname=fileloc+str(56*3+i)+".csv"
    np.savetxt(fname,data, delimiter=',', fmt='%f')

/var/folders/1l/h9b3zb0s0ws2s7d3pj0yyv740000gq/T/ipykernel_43175/4088890140.py:3: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=2, random_state=0).fit(fv.reshape(-1, 1))
/var/folders/1l/h9b3zb0s0ws2s7d3pj0yyv740000gq/T/ipykernel_43175/4088890140.py:3: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=2, random_state=0).fit(fv.reshape(-1, 1))
/var/folders/1l/h9b3zb0s0ws2s7d3pj0yyv740000gq/T/ipykernel_43175/4088890140.py:3: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=2, random_state=0).fit(fv.reshape(-1, 1))
/var/folders/1l/h9b3zb0s0ws2s7d3pj0yyv740000gq/T/ipykernel_43175/4088890140.py:3: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters

In [29]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB

In [30]:
from sklearn.preprocessing import MinMaxScaler
def normalizedata(X_train):
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    return X_train

In [31]:
def model1(trdata,tsdata,tract):
    model = BernoulliNB()
    model.fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1
def model2(trdata,tsdata,tract):
    model = MultinomialNB()
    model.fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1
def model3(trdata,tsdata,tract):
    model = GaussianNB()
    model.fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1
def model4(trdata,tsdata,tract):
    model = DecisionTreeClassifier()
    model.fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1
def model5(trdata,tsdata,tract):
    model = DecisionTreeClassifier(criterion='entropy')
    model.fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1

In [32]:
from sklearn.model_selection import KFold
kf = KFold(5)

In [33]:
for i in range(1,225):
    print(i)
    fname=fileloc+str(i)+'.csv'
    data=np.genfromtxt(fname,delimiter=',')
    X=data[:,0:-1]
    Y=data[:,-1]
    in1=np.where(Y>0)
    Y[in1[0]]=1
    X=normalizedata(X)
    predv=np.zeros((np.shape(X)[0],6))
    predv1=np.zeros((np.shape(X)[0],11))
    for train_index, test_index in kf.split(X):
        trdata=X[train_index,:]
        tsdata=X[test_index,:]
        tract=Y[train_index]
        tsact=Y[test_index]
        predv[test_index,0],predv1[test_index,0:2]=model1(trdata,tsdata,tract)
        predv[test_index,1],predv1[test_index,2:4]=model2(trdata,tsdata,tract)
        predv[test_index,2],predv1[test_index,4:6]=model3(trdata,tsdata,tract)
        predv[test_index,3],predv1[test_index,6:8]=model4(trdata,tsdata,tract)
        predv[test_index,4],predv1[test_index,8:10]=model4(trdata,tsdata,tract)
        predv[test_index,5]=tsact
        predv1[test_index,10]=tsact
    fname=fileloc+'pred'+str(i)+".csv"
    np.savetxt(fname,predv, delimiter=',', fmt='%f')    
    fname=fileloc+'predv'+str(i)+".csv"
    np.savetxt(fname,predv1, delimiter=',', fmt='%f')   

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224


In [34]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [35]:
accv=np.zeros((224,5))
fmea=np.zeros((224,5))
for i in range(1,225):
    fname=fileloc+'pred'+str(i)+".csv"
    data=np.genfromtxt(fname,delimiter=',')
    for j in range(0,5):
        accv[i-1,j]=accuracy_score(data[:,-1],data[:,j])
        fmea[i-1,j]=f1_score(data[:,-1],data[:,j])
fname=fileloc+'acc.csv'
np.savetxt(fname,accv*100, delimiter=',', fmt='%f')    
fname=fileloc+'fmea.csv'
np.savetxt(fname,fmea, delimiter=',', fmt='%f')         

In [36]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
rocv=np.zeros((224,5))
for i in range(1,225):
    fname= fileloc+'predv'+str(i)+'.csv'
    data=np.genfromtxt(fname,delimiter=',')
    y1=data[:,-1]
    for j in range(0,5):
        b=data[:,2*j+1]
        fpr, tpr, _ = roc_curve(y1, b)
        rocv[i-1,j]=auc(fpr, tpr)
fname=fileloc+'auc.csv'    
np.savetxt(fname,rocv, delimiter=',', fmt='%f')          

In [37]:
from matplotlib import pyplot
pyplot.boxplot(accv*100,labels=['BNB','MNB','GNB','DT','DTE'])
pyplot.grid(True)
pyplot.ylabel('Accuracy')
fna=fileloc+"acc.png"
pyplot.savefig(fna)
pyplot.close()

In [38]:
pvalue=np.zeros((5,5))
for i in range(0,5):
    for j in range(0,5):
        w,pvalue[i,j]=stats.ranksums(accv[:,i],accv[:,j])

In [39]:
pvalue

array([[1.        , 0.14854472, 0.13076168, 0.34627422, 0.4724405 ],
       [0.14854472, 1.        , 0.00371039, 0.00780905, 0.01631204],
       [0.13076168, 0.00371039, 1.        , 0.8020497 , 0.60637325],
       [0.34627422, 0.00780905, 0.8020497 , 1.        , 0.7586479 ],
       [0.4724405 , 0.01631204, 0.60637325, 0.7586479 , 1.        ]])

In [40]:
fname=fileloc+'pval_tec.csv'    
np.savetxt(fname,pvalue, delimiter=',', fmt='%f')    

In [41]:
np.shape(accv)

(224, 5)

In [42]:
acvf=np.zeros((56*5,4))
for i in range(0,4):
    for j in range(0,56):
        acvf[5*j:5*(j+1),i]=accv[56*i+j,:]        

In [43]:
from matplotlib import pyplot
pyplot.boxplot(acvf*100,labels=['AF','SIGF','PCA','GINI'])
pyplot.grid(True)
pyplot.ylabel('Accuracy')
fna=fileloc+"acc_fst.png"
pyplot.savefig(fna)
pyplot.close()

In [44]:
pvalue=np.zeros((4,4))
for i in range(0,4):
    for j in range(0,4):
        w,pvalue[i,j]=stats.ranksums(acvf[:,i],acvf[:,j])

In [45]:
fname=fileloc+'pval_fst.csv'    
np.savetxt(fname,pvalue, delimiter=',', fmt='%f')  